<p align="center">
<img src="https://github.com/datacamp/string-manipulation-in-sql-live-session/blob/master/assets/datacamp.svg?raw=True" alt = "DataCamp icon" width="50%">
</p>
<br><br>

## **PostgreSQL Live Training Template**

Welcome to the DataCamp "String Manipulation in SQL", where we will be covering a deep dive into a variety of techiques you can use to manipulate sting data in your SQL queries. Through a hands on coding session you will learn how to use built-in functions [...] to answer our real world business questions.

In today's notebook, you will learn:

- Explore a dataset in a SQL database by examining table structure and data types
- Converting or casting common data types
- How to manipulate string data using built-in string and character functions
- Create temporary tables and use them to simplify complex queries
- Learn how to use temporary tables to create new fields from unstructured string data
- Create user defined functions that can be used to simplify data manipulation tasks

## **The Dataset**

This data is taken from a .csv file called "short_term_rentals". As today's session will focus on creating queries. It contains the following columns.

- `id`: Unique identifier for rental ID
- `issue_date`: Date of Issue for the Short Term Rental certificate
- `status`: Application status.
- `location`: Rental Location
- `property_type`: Description of the building property type.
- `property_type_additional_description`: Additional building property type information.
- `condo_association`: Is the Short-Term Rental Unit part of a condominium association?
- `total_bedrooms`: Total number of legal bedrooms in the short-term rental unit
- `rented_bedrooms`: Number of legal bedrooms to be rented as part of this application.
- `maximum_renter_capacity`: Maximum number of people to which the short term rental will be rented.
- `kitchen`: Will the kitchen be available to the renter?
- `bathrooms`: Number of bathrooms available to the renter.
- `all_rental_services`: A single column list of all rental services used by this property.



## **Setting up PostgreSQL**

In [0]:
#@title **This block of code will install PosgreSQL**
%%capture
!wget -qO- https://www.postgresql.org/media/keys/ACCC4CF8.asc | apt-key add -
!echo "deb http://apt.postgresql.org/pub/repos/apt/ bionic-pgdg main" >/etc/apt/sources.list.d/pgdg.list
!apt -qq update
!apt -yq install postgresql-12 postgresql-client-12
!service postgresql start
# make calling psql shorter
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"  
!psql postgres -c "CREATE DATABASE root"  # now just !psql -c "..."
# load SQL extensions
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

In [0]:
#@title **This will download your data to local environment**
!wget -q https://raw.githubusercontent.com/datacamp/string-manipulation-in-sql-live-session/master/data/short_term_rentals.csv

In [3]:
#@title **This will create your table**
%%sql
-- Make sure to amend you table name, column names and types
DROP TABLE IF EXISTS rentals;
CREATE TABLE rentals(
 id int primary key,
 issue_date text,
 status text,
 location text,
 short_term_rental_type text,
 property_type text,
 property_type_additional_description text,
 condo_association text,
 total_bedrooms numeric,
 rented_bedrooms numeric,
 maximum_renter_capacity numeric,
 kitchen text,
 bathrooms numeric,
 all_rental_services text
);
COPY rentals
-- Make sure to point to correct file and delimiter 
FROM '/content/short_term_rentals.csv' DELIMITER ',' CSV HEADER;

 * postgresql+psycopg2://@/postgres


""


# **Exploring the Dataset**

Now that we have PostgreSQL installed and our dataset loaded, let's take a closer look at the data that we'll be working with. 

In [4]:
%%sql

-- View the entire dataset

SELECT *
FROM rentals 

 * postgresql+psycopg2://@/postgres


,id,issue_date,status,location,short_term_rental_type,property_type,property_type_additional_description,condo_association,total_bedrooms,rented_bedrooms,maximum_renter_capacity,kitchen,bathrooms,all_rental_services
0,13727,March 27 2018,Complete,"65 Pearl St\nCambridge, MA 02139\n(42.363228, ...",This application is for an OPERATOR-OCCUPIED s...,Single-family detached dwelling,None,No,3,2,4,Yes,1,"Airbnb, HomeAway"
1,14766,April 26 2018,Complete,"34 Essex St\nCambridge, MA 02139\n(42.366748, ...",This application is for an OWNER-ADJACENT shor...,Three-family structure,None,No,1,1,2,Yes,1,Airbnb
2,13941,January 13 2020,Complete,"24 Corporal Mcternan St\nCambridge, MA 02139\n...",This application is for an OPERATOR-OCCUPIED s...,Other,Park View Cooperative is a 12 unit cooperative...,No,1,1,2,Yes,1,Airbnb
3,54029,November 27 2019,Complete,"152 Berkshire St\nCambridge, MA 02141\n(42.372...",This application is for an OPERATOR-OCCUPIED s...,Other,The unit is one of approximately 30 units in t...,Yes,1,1,4,Yes,1,"Airbnb, HomeAway, VRBO"
4,46488,July 09 2019,Complete,"26 Sacramento St\nCambridge, MA 02138\n(42.382...",This application is for an OPERATOR-OCCUPIED s...,Two-family dwelling,None,No,4,4,6,Yes,2,Airbnb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,13856,April 05 2019,Complete,"53 Ellery St\nCambridge, MA 02138\n(42.372064,...",This application is for an OPERATOR-OCCUPIED s...,Three-family structure,None,Yes,3,3,5,Yes,3,Airbnb
277,48801,January 16 2020,Complete,"42 Bowdoin St\nCambridge, MA 02138\n(42.383883...",This application is for an OPERATOR-OCCUPIED s...,Two-family dwelling,None,Yes,5,2,4,Yes,1,Airbnb
278,31371,April 08 2019,Complete,"184 Green St\nCambridge, MA 02139\n(42.363624,...",This application is for an OPERATOR-OCCUPIED s...,Three-family structure,None,Yes,3,3,6,Yes,1,Airbnb
279,44924,May 09 2019,Complete,"100 Auburn St\nCambridge, MA 02139\n(42.362472...",This application is for an OPERATOR-OCCUPIED s...,Three-family structure,None,No,3,2,4,Yes,1,Airbnb


A few of things that you should notice from what we've seen so far.

1. **Looks like we have a date/time value `issue_date` that is stored as a `text` field rather than a date/time value.** This will likely cause unexpected results in our queries. For example:

In [0]:
%%sql

-- Sort the dataset by issue_date

SELECT *
FROM rentals
ORDER BY issue_date
LIMIT 5  

What did you notice about the results?  

Let's just confirm that the `issue_date` is indeed being sorted incorrectly by running the same query but sorting `DESCENDING` instead.

In [0]:
%%sql

-- Sort the dataset by issue_date

SELECT *
FROM rentals
ORDER BY issue_date DESC
LIMIT 5   

2. **The `all_rental_services` field contains a comma separated list of rental services.** This will mean will need to do some manipulation before doing any aggregation or analysis by rental service. 

3. **The `location` field contains all the address parts in a single column rather than individual fields like street address, city, state and postal code.** It also looks like there are lat/lon coordinates stored in this column as well which could become handy later if we want to visualize where the rentals are located across Cambridge.  For example:



In [0]:
%%sql

-- Show the ID,  location and all_rental_services columns

SELECT id, location, all_rental_services
FROM rentals
LIMIT 5   

## **Q&A**

# **Getting our data analysis-ready**

Looking more closely at the Short Term Rentals dataset, there are at least three adjustments that need to be made to the data before we can get some meaningful insights.

_Data type problems:_

- **Problem 1**: First, the `issue_date` column needs to be converted to a date/time value before we can look at timeseries data.

- **Problem 2**: And finally, we'll need split the `all_rental_serivces` column so we can know who our competitors are in the city before we launch the pilot.

<br>

_Text/categorical data problems:_

- **Problem 3**: Next, to get any aggregated insights about where in the city rental applications are most prevalent, we'll have to break apart the `location` column into it component address fields.

- **Problem 4**: Looking at the rental services in our dataset there are several that only have a single listing and we'll want to combine all these services into a single `'other'` category.


## **Our to do list:**

_Data type problems:_

- **Task 1**: Convert `issue_date` from `text` data type to `date`
- **Task 2**: Split the `all_rental_services` values into individual services

<br>

_Text/categorical data problems:_

- **Task 3**: Parse the `location` column into component address fields (i.e., zip code)
- **Task 4**: Create an `'other'` category as a "catch-all" for outlier rental services

<br>

_Data range problems:_

<br>

_Dealing with missing data:_

<br>

_Is that all though?_


## **Task 1: Converting data from one type to another using `CAST()`**

Let's get started by converting the `issue_date` to a date value.  Looking at the data as it's currently stored in the database, it appears that the column stores the month, day and year that the rental certification was issue but it's stored as a `TEXT` data type. 

To convert columns from one data type to another in PostgreSQL, we can use the `CAST()` function as below here:

``
SELECT cast(column_name as new_type) 
FROM table;
``

Since `issue_date` does not contain time, we can convert it to DATE instead of TIMESTAMP.

In [0]:
%%sql

-- Convert issue_date to a date

SELECT id, cast(issue_date AS date)
FROM rentals 
LIMIT 5


Now if we try to sort the dataset by the converted `issue_date` field, let's see if we get expected results.

In [0]:
%%sql

-- Convert issue_date to a date and sort ascending

SELECT id, cast(issue_date AS date)
FROM rentals 
--ORDER BY cast(issue_date AS date)
ORDER BY cast(issue_date AS date) DESC
LIMIT 5


## **Task 2: Extracting string data using `SUBSTRING()` and `POSITION()`**

Great work!  Now let's see what we can do with the `location` field.  Ideally, we would have columns for each address part including street address, postal code, etc. In order to do this we'll need to parse the `location` field into these parts. 

To divide strings (or extract substrings) in PostgreSQL, we can use `SUBSTRING`(column_name, substring_start_position, substring_end_position)` where:

> `column_name` is the name of the column being processed
<br> `substring_start_position` is the numbered position of the first string of the substring being extracted.
<br> `substring_end_position` is the numbered position of the substring of the last string of the substring being extracted.

A common value from the location column has the format:

> `street_address \n city_state_zip \n lat_long`
<br> <br>For example: `65 Pearl St\nCambridge, MA 02139\n(42.363228, 42.36322)`

To divide `location` into substrings, will divide it based on the position of the break-line character `\n` by using the `POSITION(character IN column_name)` function which finds the numbered position of a character in a string column, as well as the `chr(10)` function which returns `\n`.

In [0]:
%%sql

-- Use substring() and position() functions to parse the location into various address values

SELECT substring(location, 0, position(chr(10) IN location)) AS street_address,
       substring(location, position(chr(10) IN location)+1, 19) AS city_state_zip,
       substring(location, position('(' IN location), position(')' IN location)) AS lat_lon
FROM rentals
LIMIT 5;


## **Extracting string data using `split_part()`**

The `split_part()` function parses string data based on a specified delimiter into an `ARRAY` and allows access to each array index by specifying the index value as a parameter in the function.  The query above can be rewritten using the `split_part()` function which as you'll see actually simplifies the query.

![split_part graphic](https://user-images.githubusercontent.com/48436758/83009911-c3ecf600-a017-11ea-81c6-2ae5b179fcc1.png)


In [0]:
%%sql

-- Use split_part() to parse the location into various address values

SELECT split_part(location, chr(10), 1) AS street_address, 
       split_part(location, chr(10), 2) AS city_state_zip, 
       split_part(location,chr(10), 3) AS lat_lon
FROM rentals
LIMIT 5;


## **Using `ARRAYS` to manipulate strings stored as comma-separated-values**

In [0]:
%%sql

-- Use string_to_array() to convert the all_rental_services from a comma-separated-list to an array

SELECT id, string_to_array(all_rental_services,',')
FROM rentals
LIMIT 5;


Now let's look closer at the result.  The comma-separated-list has been split into an `ARRAY` for each rental.  However, it appears that doing this produces `ARRAYS` with variable lenght which are going to be hard to work with later as we begin our analysis.

Ideally, we'd be able to have an indicator for each service to determine whether a rental was using that service.  Let's see if we can figure out how many service sare stored in the `all_rental_services` field.

PostgreSQL has a function called `unnest()` that will take an array and create unique rows for each index. Let's see what happens when we use the `unnest()` function on the result of `string_to_array()` for the first rental id 13727.

In [0]:
%%sql

-- Nest the string_to_array() function in unnest() to transpose the array into multiple unique rows.

SELECT id, unnest(string_to_array(all_rental_services,','))
FROM rentals
WHERE id = 13727;


The result of the `unnest()` function ened up creating two rows - one for Airbnb and one for HomeAway - both with id 13727.

Now we can get use these new skills to determine how many rentals are registered for each service by nesting the query above as a subquery and performing an aggregation to `COUNT()` the number of rental ids for each service. 

In [0]:
%%sql

-- Use the results from the unnest() function in the query

--SELECT count(id), services
SELECT count(id), trim(services)
FROM
(
    SELECT id, unnest(string_to_array(all_rental_services,','))
    FROM rentals
 ) as dt(id, services) 
-- GROUP BY services
GROUP BY trim(services)
ORDER BY count(id) DESC;
       


## **What Have We Learned About the Short Term Rentals Data?**

So far, we have discovered:

*   There rentals dataset contains data from March, 2018 through April, 2020.
*   The short term rentals market has about 22 different services.





## **Q&A**

# **Techniques for query simplification and code reuse**

Sometimes when you encounter data that needs to be transformed or manipulated before being usable in queries, you may decide that creating a temporary table or user-defined function with the result of your data manipulation will simplify your data analysis queries or allow for code reuse in multiple queries. 

Let's explore some techniques by first looking at how to use temporary tables

## **Creating a temporary table with our rental services data**

A temporary table in PostgeSQL is equivalent to a permanent disk-based table created in a database but only exists during the duration of the database session. PostgreSQL will automatically drop the table at the end of the session or 

>`CREATE TEMP[ORARY] TABLE temp_table(
   ...
);`

In [5]:
%%sql
DROP TABLE IF EXISTS rental_services;
-- Convert issue_date to a date and sort ascending
CREATE TEMP TABLE rental_services AS
  SELECT id, unnest(string_to_array(all_rental_services,',')) AS service
  FROM rentals;
SELECT *
FROM rental_services
LIMIT 5;
       


 * postgresql+psycopg2://@/postgres


,id,service
0,13727,Airbnb
1,13727,HomeAway
2,14766,Airbnb
3,13941,Airbnb
4,54029,Airbnb


In [6]:
%%sql
DROP TABLE IF EXISTS address;
-- Convert issue_date to a date and sort ascending
CREATE TEMP TABLE address AS
  SELECT id, split_part(location, chr(10), 1) AS street_address, 
       right(split_part(location, chr(10), 2), 5) AS zip, 
       split_part(location,chr(10), 3) AS lat_lon
  FROM rentals;
SELECT *
FROM address
LIMIT 5;


 * postgresql+psycopg2://@/postgres


,id,street_address,zip,lat_lon
0,13727,65 Pearl St,02139,"(42.363228, -71.104202)"
1,14766,34 Essex St,02139,"(42.366748, -71.101654)"
2,13941,24 Corporal Mcternan St,02139,"(42.361299, -71.108347)"
3,54029,152 Berkshire St,02141,"(42.372124, -71.089837)"
4,46488,26 Sacramento St,02138,"(42.382554, -71.117114)"


## **Using temporary tables to simplify complex queries**

Once the temporary tables are created, your queries become much simpler and easier to manage, and you have the ability to run multiple queries against the temporary table which streamlines reuse.

We will use an `INNER JOIN` to combine the results from the newly created temporary tables `address` and `rental_services`.  As a review, an `INNER JOIN` will return rows that are in both table A and table B as highlighted in the image below. 

![sql joins](https://user-images.githubusercontent.com/48436758/83012005-1b409580-a01b-11ea-99b8-7bb73a822fb7.png)

For example:

> SELECT table_1_alias.* , table_2_alias.*<br> 
FROM table_1 AS table_1_alias INNER JOIN table_2 AS table_2_alias 
ON table_1_alias.common_column = table_2_alias.common_column;

In [0]:
%%sql
SELECT a.*, s.*
FROM address AS a INNER JOIN rental_services AS s ON a.id=s.id;

In [0]:
%%sql
SELECT a.zip, s.service, count(a.id) as total
FROM address AS a INNER JOIN rental_services AS s ON a.id=s.id
GROUP BY a.zip, s.service
ORDER BY zip, service;

## **User-defined functions to create reusable code**

A user-defined function is the PostgeSQL equivalent of a stored procedure where you can bundle several SQL queries and statements together into a single package using the `CREATE FUNCTION` command. 

>`CREATE [OR REPLACE] FUNCTION function_name(p1 type, p2 type)`<br>
`RETURNS type AS $$`<br>
`BEGIN`<br>
`-- logic`<br>
`END;`<br>
`$$ LANGUAGE language_name;`

In [0]:
%%sql
CREATE OR REPLACE FUNCTION getZipCode (rentalID int)
RETURNS varchar(5) AS $$
DECLARE zip varchar(5);
BEGIN
  SELECT right(split_part(location, chr(10), 2), 5) INTO zip
  FROM rentals
  WHERE id = rentalID;

  RETURN zip;
END;
$$ LANGUAGE plpgsql;

SELECT id, location, getZipCode(id) 
FROM rentals;


# **Case study: Analyzing competitors and the best locations for short term rentals**

Let's now use our new data manipulation skills to dig deeper into the short term rentals data to determine if Cambridge, MA is a good spot to pilot our new short term rentals app.

First, we'll see what the growth rate has been over the past couple of years with registrations.

Next we'll see what the most popular rental services are in the town.

And finally we'll take a look at which zip codes in the city have the largest number of rental units.  

Let's get started.

In [0]:
%%sql
SELECT extract(year FROM CAST(issue_date AS DATE)) AS year, count(id) 
FROM rentals
GROUP BY year
ORDER BY year;

In [0]:
%%sql
SELECT CASE WHEN trim(service) IN('Airbnb', 'HomeAway', 'VRBO', 'FlipKey', 
                                  'Craigslist', 'Boston_Rentals', 'Couch_Surfing') 
       THEN trim(service) ELSE 'Other' END as rentalService,
      count(r.id) as total
FROM rentals AS r INNER JOIN rental_services AS s ON r.id=s.id
GROUP BY rentalService
ORDER BY total DESC;


In [0]:
%%sql
SELECT getZipCode(r.id) AS zipCode, count(r.id) as total
FROM rentals AS r INNER JOIN rental_services AS s ON r.id=s.id
GROUP BY zipCode
ORDER BY zipCode;


# **Q&A**

# **Recap and Closing**

Thank you!